### Importing some useful libraries

In [3]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# import cv2
import matplotlib.pyplot as plt

from sklearn.model_selection import train_test_split

import tensorflow as tf

from keras.models import Sequential, load_model
from keras.callbacks import EarlyStopping, ModelCheckpoint
from keras import layers
from keras.layers import *
from keras.utils import np_utils

from tqdm import tqdm

### 1. Emnist Balanced Train

### Importing CSV

In [6]:
train_df = pd.read_csv('C:/Users/LENOVO/Downloads/DatasetDisabilitas.csv', header=None)
train_df.head()

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16
0,disability_type,skills_one,skills_two,company,city,province,job_type,position,is_hearing_impairment,is_visual_impairment,is_physical_disability,is_all_types_of_disabilities,is_neurodiversity,is_mobility_impairment,is_speech_impairment,is_cognitive_disability,is_learning_disability
1,Visual Impairment,Java,Python,TechSol Solutions,Jakarta,Jakarta Raya,Full-time,Software Engineer,0,1,0,0,0,0,0,0,0
2,Hearing Impairment,Graphic Design,Adobe Photoshop,Artistic Creations,Bandung,West Java,Part-time,Graphic Designer,1,0,0,0,0,0,0,0,0
3,Physical Disability,Mechanical Engineering,AutoCAD,MegaTech Industries,Surabaya,Jawa Timur,Full-time,Mechanical Engineer,0,0,1,0,0,0,0,0,0
4,Cognitive Disability,Data Analysis,Microsoft Excel,Analytica Solutions,Yogyakarta,Special Region of Yogyakarta,Contract,Data Analyst,0,0,0,0,0,0,0,1,0


In [7]:
train_df.shape

(559, 17)

### Split train dataframe into X & y

In [8]:
X_train = train_df.loc[:, 1:]
y_train = train_df.loc[:, 0]

X_train.shape, y_train.shape

((559, 16), (559,))

In [9]:
X_train.head()

,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16
0,skills_one,skills_two,company,city,province,job_type,position,is_hearing_impairment,is_visual_impairment,is_physical_disability,is_all_types_of_disabilities,is_neurodiversity,is_mobility_impairment,is_speech_impairment,is_cognitive_disability,is_learning_disability
1,Java,Python,TechSol Solutions,Jakarta,Jakarta Raya,Full-time,Software Engineer,0,1,0,0,0,0,0,0,0
2,Graphic Design,Adobe Photoshop,Artistic Creations,Bandung,West Java,Part-time,Graphic Designer,1,0,0,0,0,0,0,0,0
3,Mechanical Engineering,AutoCAD,MegaTech Industries,Surabaya,Jawa Timur,Full-time,Mechanical Engineer,0,0,1,0,0,0,0,0,0
4,Data Analysis,Microsoft Excel,Analytica Solutions,Yogyakarta,Special Region of Yogyakarta,Contract,Data Analyst,0,0,0,0,0,0,0,1,0


In [10]:
y_train.head()

0         disability_type
1       Visual Impairment
2      Hearing Impairment
3     Physical Disability
4    Cognitive Disability
Name: 0, dtype: object

### Create label dictionary

In [21]:
label_map = pd.read_csv("C:/Users/LENOVO/Downloads/DatasetDisabilitas.csv", 
                        delimiter = ' ', 
                        index_col=0, 
                        header=None, 
                        squeeze=True)
label_map.head()

C:\Users\LENOVO\AppData\Local\Temp/ipykernel_28900/2698011649.py:1: FutureWarning: The squeeze argument has been deprecated and will be removed in a future version. Append .squeeze("columns") to the call to squeeze.


  label_map = pd.read_csv("C:/Users/LENOVO/Downloads/DatasetDisabilitas.txt",


FileNotFoundError: [Errno 2] No such file or directory: 'C:/Users/LENOVO/Downloads/DatasetDisabilitas.txt'

In [53]:
label_dictionary = {}
for index, label in enumerate(label_map):
    label_dictionary[index] = chr(label)

label_dictionary

NameError: name 'label_map' is not defined

### Visualize sample data

In [54]:
# Sample entry number 42
sample_image = X_train.iloc[42]
sample_label = y_train.iloc[42]
sample_image.shape, sample_label

((16,), 'Cognitive Disability')

In [55]:
print("Label entry 42:", label_dictionary[sample_label])
plt.imshow(sample_image.values.reshape(28, 28), cmap=plt.cm.gray)
plt.show()

KeyError: 'Cognitive Disability'

Next we want to apply reshape_and_rotate to all images in X_train

In [57]:
X_train = np.apply_along_axis(reshape_and_rotate, 1, X_train.values)
X_train.shape

ValueError: cannot reshape array of size 16 into shape (28,28)

## Visualize more sample

In [58]:
sample_image = X_train[42]
sample_label = y_train.iloc[42]
print("Label entry 42:", label_dictionary[sample_label])
plt.imshow(sample_image.reshape(28, 28), cmap=plt.cm.gray)
plt.show()


for i in range(100, 106):
    plt.subplot(390 + (i+1))
    plt.imshow(X_train[i], cmap=plt.cm.gray)
    plt.title(label_dictionary[y_train[i]])

KeyError: 42

Looks good

### Normalize Data

In [59]:
X_train = X_train.astype('float32') / 255

ValueError: could not convert string to float: 'Project Management'

### One Hot Encode Label

In [60]:
number_of_classes = y_train.nunique()
number_of_classes

15

In [61]:
y_train = np_utils.to_categorical(y_train, number_of_classes)
y_train.shape

ValueError: invalid literal for int() with base 10: 'Learning Disability'

In [62]:
y_train

171                                  Learning Disability
130                                    Speech Impairment
32                                   Mobility Impairment
373                                    Visual Impairment
139                                   Hearing Impairment
                             ...                        
197                         Neurodiversity (Dyscalculia)
112                                    Speech Impairment
362    Neurodiversity (Attention Deficit Hyperactivit...
433                                    Speech Impairment
288                                    Visual Impairment
Name: 0, Length: 503, dtype: object

In [63]:
# Reshape to fit model input shape
# Tensorflow (batch, width, height, channels)
X_train = X_train.reshape(-1, 28, 28, 1)
X_train.shape

AttributeError: 'DataFrame' object has no attribute 'reshape'

In [64]:
# Split 10% validation 
X_train, X_val, y_train, y_val = train_test_split(X_train, 
                                                  y_train, 
                                                  test_size= 0.1, 
                                                  random_state=88)

### Define model

In [65]:
model = Sequential()

model.add(layers.Conv2D(filters=32, kernel_size=(5,5), padding='same', activation='relu', input_shape=(28, 28, 1)))
model.add(layers.MaxPool2D(strides=2))
model.add(layers.Conv2D(filters=48, kernel_size=(5,5), padding='valid', activation='relu'))
model.add(layers.MaxPool2D(strides=2))
model.add(layers.Flatten())
model.add(layers.Dense(256, activation='relu'))
model.add(layers.Dense(84, activation='relu'))
model.add(layers.Dense(number_of_classes, activation='softmax'))
model.compile(loss='categorical_crossentropy', optimizer='adam',
 metrics=['accuracy'])


model.summary()

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d_2 (Conv2D)           (None, 28, 28, 32)        832       
                                                                 
 max_pooling2d_2 (MaxPooling  (None, 14, 14, 32)       0         
 2D)                                                             
                                                                 
 conv2d_3 (Conv2D)           (None, 10, 10, 48)        38448     
                                                                 
 max_pooling2d_3 (MaxPooling  (None, 5, 5, 48)         0         
 2D)                                                             
                                                                 
 flatten_1 (Flatten)         (None, 1200)              0         
                                                                 
 dense_3 (Dense)             (None, 256)              

In [66]:
optimizer_name = 'adam'

model.compile(loss='categorical_crossentropy', optimizer=optimizer_name, metrics=['accuracy'])

early_stopping = EarlyStopping(monitor='val_loss', patience=5, verbose=1, mode='min')
# mcp_save = ModelCheckpoint('my_model.h5', save_best_only=True, monitor='val_loss', verbose=1, mode='auto')

In [67]:
history = model.fit(X_train,
                    y_train, 
                    epochs=30, 
                    batch_size=32, 
                    verbose=1, 
                    validation_split=0.1,
                    callbacks=[early_stopping, mcp_save])

C:\Python310\lib\site-packages\keras\engine\data_adapter.py:1700: FutureWarning: The behavior of `series[i:j]` with an integer-dtype index is deprecated. In a future version, this will be treated as *label-based* indexing, consistent with e.g. `series[i]` lookups. To retain the old behavior, use `series.iloc[i:j]`. To get the future behavior, use `series.loc[i:j]`.
  return t[start:end]


ValueError: Failed to convert a NumPy array to a Tensor (Unsupported object type float).

In [68]:
# plot accuracy and loss
def plotgraph(epochs, acc, val_acc):
    # Plot training & validation accuracy values
    plt.plot(epochs, acc, 'b')
    plt.plot(epochs, val_acc, 'r')
    plt.title('Model accuracy')
    plt.ylabel('Accuracy')
    plt.xlabel('Epoch')
    plt.legend(['Train', 'Val'], loc='upper left')
    plt.show()
    
acc = history.history['accuracy']
val_acc = history.history['val_accuracy']
loss = history.history['loss']
val_loss = history.history['val_loss']
epochs = range(1,len(acc)+1)

NameError: name 'history' is not defined

#### Test accuracy

In [73]:
model.evaluate(X_val, y_val)

ValueError: Failed to convert a NumPy array to a Tensor (Unsupported object type float).

### Load test dataset & preprocess image like how we did to train dataset

In [74]:
test_df = pd.read_csv('C:/Users/LENOVO/Downloads/DatasetDisabilitas.csv', header=None)
test_df.shape

(559, 17)

In [75]:
test_df.describe()

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16
count,559,559,511,559,559,559,559,559,559,559,559,559,559,559,559,559,559
unique,15,93,163,325,85,39,11,131,3,3,3,3,3,3,3,3,3
top,Physical Disability,No Specific Skill,Python,RST Events,Jakarta,West Java,Full-time,Data Analyst,0,0,0,0,0,0,0,0,0
freq,96,44,30,8,65,150,126,35,482,468,462,522,412,541,532,526,519


In [76]:
X_test = test_df.loc[:, 1:]
y_test = test_df.loc[:, 0]

X_test.shape, y_test.shape

((559, 16), (559,))

In [77]:
X_test = np.apply_along_axis(reshape_and_rotate, 1, X_test.values)
y_test = np_utils.to_categorical(y_test, number_of_classes)

X_test.shape, y_test.shape

ValueError: cannot reshape array of size 16 into shape (28,28)

In [78]:
X_test = X_test.astype('float32') / 255

ValueError: could not convert string to float: 'skills_one'

In [79]:
X_test = X_test.reshape(-1, 28, 28, 1)
X_test.shape

AttributeError: 'DataFrame' object has no attribute 'reshape'

In [80]:
model.evaluate(X_test, y_test)

ValueError: Failed to convert a NumPy array to a Tensor (Unsupported object type float).